In [ ]:
import config
import asyncio
import ethical_control
import security_privacy_check
import layout
import gpio_communication
from langchain_core.prompts import ChatPromptTemplate
import tkinter as tk
import time
import math
import sys

In [ ]:
# If this is running using ipykernel, import nest_asyncio and apply it
if 'ipykernel' in sys.modules:
    import nest_asyncio
    nest_asyncio.apply()

In [ ]:
# Set the initial start, end and action positions
start_location = 'Charging Station'
goal_location = 'Charging Station'
action_command = 'None'

In [ ]:
# Prompt template
system_template_destination = ("You are presented with an instruction destined for a robot. You need to return a"
                               " destination location/utility, an action, or both. The format of your response should"
                               " be given as \"Destination: ..., Action:...\" where \"...\" is replaced by the desired"
                               " destination and action. If no destination or action is found in the command, replace"
                               " \"...\" with \"None\". Your responses are limited to the locations and actions in the"
                               " lists provided.")

In [ ]:
# Define the prompt
prompt_template_destination = ChatPromptTemplate.from_messages([("system", system_template_destination), ("user",
                                                                                                          "{text}")])

In [ ]:
# Define the chain
chain_destination = prompt_template_destination | config.model | config.parser

In [ ]:
# Define the data for directions
directions_data = {
    "start_location": start_location,
    "goal_location": goal_location,
    "graph": layout.graph,
    "heuristic": layout.h,
    "action": action_command
    }

In [ ]:
# Function to find the route
async def process_command_route():
    response = await (layout.a_star_search(layout.graph, directions_data["start_location"],
                                           directions_data["goal_location"], layout.h))
    print("Route found to be: ", response)
    return response

In [ ]:
# Function to find the destination and action
def interpret_command(command):
    # Add the list of possible locations and actions to the command
    updated_command = (command + " The list containing available locations/utilities is: " + str(layout.h) +
                       " The list containing available actions is: " + str(layout.actions))
    
    # Invoke the LLM to generate the response
    response = chain_destination.invoke({"text": updated_command})
    
    # Extract the location from the response
    destination_start = response.find(":") + 2
    destination_end = response.find(",")    
    destination = response[destination_start:destination_end].strip()
    
    # Extract the action from the command
    action_start = response.find(":", destination_end + 1) + 2
    action = response[action_start:]
    
    print("Action was recorded as: ", action)
    
    # Check if we have a valid location
    try:
        location = layout.h[destination]
        directions_data["goal_location"] = destination
        directions_data["action"] = action
        return True
    except KeyError:
        print("Location not identified!\n")
        # If location does not exist, set the goal location to be the start location
        directions_data["goal_location"] =  directions_data["start_location"]
        return False

In [ ]:
# Function to create lists of directions and distances
def directions_list_create(location_route):
    # Set the current location to be the start_location
    current_location = directions_data["start_location"]
    
    # Create lists to store directions and distances
    directions_list = []
    distance_list = []
    
    # Look through the location route list
    for item in location_route:
        # Ignore start location
        if item == directions_data["start_location"]:
            continue
            
        # Look through the list of available locations from the current location
        for location in layout.graph[current_location]:
            if location[0] == item:
                directions_list.append(location[2])
                distance_list.append(location[1])
        current_location = item
        
    print("Directions: ", directions_list)
    print("Distance: ", distance_list)
    return directions_list, distance_list

In [ ]:
# Function to control the motor towards its destination
async def motion_control(route, distances, locations):
    
    # Counter for moving through the route
    counter = 0
    
    # Loop through each item in route and command motors to move in given directions
    for item in route:
        print("Moving ", item)
        if item == "forward":
            await asyncio.gather(gpio_communication.motor_forward(distances[counter]))

        elif item == "back":
            await asyncio.gather(gpio_communication.motor_backward(distances[counter]))
            
        elif item == "left":
            await asyncio.gather(gpio_communication.motor_left(distances[counter]))
        
        elif item == "right":
            await asyncio.gather(gpio_communication.motor_right(distances[counter]))
            
        elif item == "diagonally forward and left":
            await (asyncio.gather(gpio_communication.motor_forward(distances[counter]),
                                  gpio_communication.motor_left(distances[counter])))
        
        elif item == "diagonally forward and right":
            await (asyncio.gather(gpio_communication.motor_forward(distances[counter]),
                                  gpio_communication.motor_right(distances[counter])))
            
        elif item == "diagonally back and right":
            await (asyncio.gather(gpio_communication.motor_backward(distances[counter]),
                                  gpio_communication.motor_right(distances[counter])))
            
        elif item == "diagonally back and left":
            await (asyncio.gather(gpio_communication.motor_backward(distances[counter]),
                                  gpio_communication.motor_left(distances[counter])))
        
        else:
            print("Unknown command for motor control")
            
        # Stop motors after given move
        gpio_communication.motor_stop()
        
        # Quit this loop if the stop command has been given. Do not do the next move in the route list
        if config.global_stop == True:
            break           
        
        # Update current position
        print("Successfully moved from ", directions_data['start_location'], " to ", locations[counter])
        print("Setting current location to ", locations[counter])
        directions_data["start_location"] = locations[counter]
        counter = counter + 1 

In [ ]:
# Function to action the command received from the user        
async def action_command(command):
    
    # Show we have an active task
    config.task_active = True
        
    # Update goal_location and action    
    if not interpret_command(command.title()):
        # Return if goal location invalid
        return
    
    # If already at goal location, no need to move    
    if directions_data["start_location"] == directions_data["goal_location"]:
        print("Already at location ", directions_data["goal_location"])
        return
        
    # State where we start and where we end    
    print("Start location is: ", directions_data["start_location"])
    print("Goal location is: ", directions_data["goal_location"])
    
    # Get the route we'll take
    directions = await process_command_route()
    if directions == "That location is unknown.":
        print("location not found. Please try again.")
        return
    
    # Get specific directions and distances from the given route
    directions_list, distance_list = directions_list_create(directions)

    # Simulate motor response
    await motion_control(directions_list, distance_list, directions)
    
    # Update start location
    directions_data["start_location"] = directions_data["goal_location"]
    
    # Mark task as complete
    config.task_active = False

In [ ]:
# Function to get input from the user
async def input_loop(queue, gui_app):
    import test
    # Initialise a counter for counting tests in testing mode
    test_loop = 0
    
    while not config.global_quit:
        # Check if we are running tests
        if config.test_type == 1:
            if test_loop<config.test_1a_duration:
                user_input = await test.standard_inputs()
                await asyncio.sleep(config.test_frequency)
                test_loop = test_loop+1
            elif config.test_1a_duration <= test_loop < config.test_1b_duration:
                if config.test_1a_duration == test_loop:
                    print("\n\n***********************Now Testing Commands to infer location**********************\n\n")
                user_input = await test.input_for_inference()
                await asyncio.sleep(config.test_frequency)
                test_loop = test_loop + 1
                # Cycle through the 4 commands
                if config.command_number<3:
                    config.command_number = config.command_number + 1
                else:
                    config.command_number = 0
            else:
                # Quit the first test once completed commands from queue
                if not config.task_active and queue.empty():
                    config.global_quit = True
                    # Reset the command number counter
                    config.command_number = 0
                else:
                    # Allow operations to continue while we wait for queue to empty
                    await asyncio.sleep(0.01)
                continue
        elif config.test_type == 2:
            user_input = await test.unethical_inputs()
            await asyncio.sleep(config.test_frequency)
            # Steps through to the next command in the list
            config.command_number = config.command_number + 1
        else:
            # Get user input without blocking the event loop
            user_input = await gui_app.get_input()
        
        # Check user is authorised to give command and remove personal data before transmission to LLM
        await security_privacy_check.check_security()

        # If user has typed quit, stop the motors and quit the program
        if user_input.lower() == 'quit':
            gpio_communication.motor_stop()
            config.global_quit = True
            break
        
        # If user has typed stop, stop the motors and clear queue    
        if user_input.lower() == 'stop':            
            gpio_communication.motor_stop()
            # Purge queue
            await ethical_control.purge_queue(queue)
            break
            
        # Check command for ethical issues. If issue found, restart loop. Otherwise, add the command to the queue
        if await ethical_control.check_ethics(user_input):
            continue
        else:
            await queue.put(user_input)

In [ ]:
# Function to process the commands in the queue
async def process_commands(queue):
    
    # Notification of queue being empty set to true initially because we do not need to notify on program launch
    empty_notified = True
    
    # Robot waiting time set to value above 60 initially to indicate it is already at charging station
    start_time = config.waiting + 1

    while not config.global_quit and not config.global_stop:
        
        # Check to see if the queue is empty and if a notification has been given. Give one if not.
        if queue.empty() and not empty_notified:
            empty_notified = True
            print("Awaiting further instructions...\n")
            start_time = math.trunc(time.time())
            await asyncio.sleep(0.01)
            continue
        # Move robot back to charge if queue is empty, a notification has been given, and it's not on charge already.
        if directions_data["start_location"] != "Charging Station":   
            if queue.empty() and empty_notified:
                if math.trunc(time.time()) - start_time < config.waiting:
                    await asyncio.sleep(0.01)
                    continue
                elif math.trunc(time.time()) - start_time == config.waiting:
                    print("Returning to charge and awaiting further instructions.\n")
                    await action_command("Charging Station")
                    print("\nAwaiting further instructions.\n")
                    continue
                
        # Retrieve the next command from the queue
        command = await queue.get()
        empty_notified = False        
        print("Processing command: ", command)
        # Action a user command

        await action_command(command)
        print("Finished processing command: ", command)
        print("\n")
        queue.task_done()

In [ ]:
class GUIApp():
    def __init__(self, queue, root):
        
        self.queue = queue
        
        # Create the command input window
        self.root = root
        self.root.geometry("600x100")
        self.root.title("Command input")
        
        # Command input cell
        self.entry = tk.Entry(self.root, width=40)
        self.entry.pack()
        self.entry.focus_set()
        
        # Setup for user to be able to hit enter or press submit button
        self.entry.bind("<Return>", self.on_enter)
        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit_input)
        self.submit_button.pack()
        
        # Setup for user to be able to hit emergency stop button
        self.stop_button = tk.Button(self.root, text="Emergency Stop!", command=self.run_emergency_stop)
        self.stop_button.pack()
        self.input_future = None
        
    # non async function to trigger emergency stop from button push    
    def run_emergency_stop(self):
        asyncio.create_task(self.emergency_stop())
    
    # Function to perform Emergency stop and clear the queue
    async def emergency_stop(self):
        gpio_communication.motor_stop()
        print("Emergency stop!")
        await ethical_control.purge_queue(self.queue)
        config.global_stop = True
    
    #Function to send the input from the user
    def submit_input(self):
        # When the button is clicked (or enter key hit), set the result of the future to the user input
        if self.input_future:
            self.input_future.set_result(self.entry.get())
            # Clear the text from the field
            self.entry.delete(0, tk.END)

    #Function to get the input from the user
    async def get_input(self):
        # Create a new future and wait for input from the GUI
        self.input_future = asyncio.get_event_loop().create_future()
        text = await self.input_future
        print("\nCommand received: ", text)
        print("\n")
        return text
    
    # Function to submit user input when 'enter' is pressed.
    def on_enter(self, event):
        self.submit_input()

In [ ]:
# Function to refresh the window periodically
async def update_tk(root):
    
    while not config.global_quit and not config.global_stop:
        # Try to update the window, if not possible (i.e. window closed), stop the motors and terminate program
        try:
            if not root.winfo_exists():
                raise tk.TclError("window destroyed")
            root.update()
        except tk.TclError:
            gpio_communication.motor_stop()
            config.global_quit = True
            break
        await asyncio.sleep(0.01)

In [ ]:
# Main function
async def main():
    # If we're doing one of the tests, print to file
    if config.test_type == 1:
        log_file = open("output_standard_commands.log", "w")
        sys.stdout = log_file
    if config.test_type == 2:
        log_file = open("output_unethical_commands.log", "w")
        sys.stdout = log_file
    
    # Set up the command queue
    queue = asyncio.Queue()    
    
    # Set up the GUI
    root = tk.Tk()
    root.attributes('-topmost', 1)
    gui_app = GUIApp(queue, root)
    
    # Run the asyncio event loop with the Tkinter main loop
    loop = asyncio.get_event_loop()
    
    while not config.global_quit:
        # Set up main concurrent tasks
        producer = asyncio.create_task(input_loop(queue, gui_app))
        consumer = asyncio.create_task(process_commands(queue))
        gui_refresh = asyncio.create_task(update_tk(root))
        ongoing_ethics = asyncio.create_task(ethical_control.ethical_triggers(queue))
        
        # Look out for producer, gui_refresh or ongoing_ethics to complete so we can cancel the other tasks
        done, pending = await (asyncio.wait([producer, consumer, gui_refresh, ongoing_ethics],
                                            return_when=asyncio.FIRST_COMPLETED))
        # Cancel other tasks if one ends
        if producer in done:
            consumer.cancel()
            gui_refresh.cancel()
            ongoing_ethics.cancel()
        elif ongoing_ethics in done:
            producer.cancel()
            consumer.cancel()
            gui_refresh.cancel()
        elif gui_refresh in done:
            producer.cancel()
            consumer.cancel()
            ongoing_ethics.cancel()
            
        # Reset global_stop command
        config.global_stop = False

    # Close the log file if we've been testing    
    if config.test_type == 1 or config.test_type ==2:
        sys.stdout=sys.__stdout__
        log_file.close()
        
    # If the command input window is still open, destroy it
    try:
        if not root.winfo_exists():
            raise tk.TclError("window destroyed")
        root.destroy()
    except tk.TclError:
        config.global_quit = True    

In [ ]:
if __name__ == "__main__":
    asyncio.run(main())